In [1]:
!kaggle datasets download -d pierremegret/dialogue-lines-of-the-simpsons

/bin/sh: kaggle: command not found


In [2]:
!head simpsons_dataset.csv

In [3]:
!unzip dialogue-lines-of-the-simpsons.zip

unzip:  cannot find or open dialogue-lines-of-the-simpsons.zip, dialogue-lines-of-the-simpsons.zip.zip or dialogue-lines-of-the-simpsons.zip.ZIP.


In [4]:
import pandas as pd

In [5]:
lines_df = pd.read_csv('simpsons_dataset.csv')
lines_df = lines_df.dropna()
characters = lines_df['raw_character_text'].values
lines = lines_df['spoken_words'].values

In [6]:
lines_df.head(20)

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...
5,Martin Prince,I don't think there's anything left to say.
6,Edna Krabappel-Flanders,Bart?
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Landlady,"Hey, hey, he Moved out this morning. He must h..."


In [7]:
lines[:1000]

array(["No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.",
       "Where's Mr. Bergstrom?",
       "I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?",
       'That life is worth living.',
       "The polls will be open from now until the end of recess. Now, just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?",
       "I don't think there's anything left to say.", 'Bart?',
       'Victory party under the slide!', 'Mr. Bergstrom! Mr. Bergstrom!',
       'Hey, hey, he Moved out this morning. He must have a new job -- he took his Copernicus costume.',
       'Do you know where I could find him?',
       "I think he's taking the next train to Capital City.",
       'The train, how like him... traditional, yet environmentally sound.',
       "Yes, and it's been the backbo

In [9]:
!pip install spacy

    100% |████████████████████████████████| 34.4MB 671kB/s ta 0:00:011   11% |███▊                            | 4.0MB 41.1MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 22.6MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 5.7MB/s ta 0:00:011
    100% |████████████████████████████████| 276kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 2.9MB 2.5MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 2.8MB/s ta 0:00:011   60% |███████████████████▌            | 1.8MB 3.4MB/s eta 0:00:01


In [10]:
import spacy
docs = []
nlp = spacy.load('en_core_web_lg')
count = 0
for doc in nlp.pipe(lines):
    docs.append(doc)

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
vectors = [doc.vector for doc in docs]

In [ ]:
lines_df['vectors'] = vectors

In [ ]:
lines_df.head()

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(vectors)

In [ ]:
def get_vectors_of_string(inp_str):
    return nlp(inp_str).vector

In [ ]:
test_vect = get_vectors_of_string("Mountain Dew or Crab Juice")

In [ ]:
neighbs = nn.kneighbors([test_vect])

In [ ]:
def find_vector(vect):
    return lines_df.apply(lambda x: x if x['vectors'] == vect else '', axis=1)

In [ ]:
neighbs[1][0]

In [ ]:
lines_df.iloc[neighbs[1][0]]

In [ ]:
def find_quotes(inp_str):
    vect = get_vectors_of_string(inp_str)
    closest_quotes = nn.kneighbors([vect])
    return lines_df.iloc[closest_quotes[1][0]]

In [ ]:
find_quotes("they're all in the hammock district")

In [ ]:
find_quotes("steamed hams")

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
!pip install tensorflow_hub

In [ ]:
def heatmap(x_labels, y_labels, values):
    fig, ax = plt.subplots()
    im = ax.imshow(values)

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(x_labels)))
    ax.set_yticks(np.arange(len(y_labels)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(x_labels)
    ax.set_yticklabels(y_labels)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", fontsize=10,
         rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(y_labels)):
        for j in range(len(x_labels)):
            text = ax.text(j, i, "%.2f"%values[i, j],
                           ha="center", va="center", color="w", 
fontsize=6)

    fig.tight_layout()
    plt.show()

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# sample text
messages = [
# Smartphones
"My phone is not good.",
"Your cellphone looks great.",

# Weather
"Will it snow tomorrow?",
"Recently a lot of hurricanes have hit the US",

# Food and health
"An apple a day, keeps the doctors away",
"Eating strawberries is healthy",
]

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

    corr = np.inner(message_embeddings_, message_embeddings_)
    print(corr)
    heatmap(messages, messages, corr)